In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools as it
from collections import Counter

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1290,tt2016940,25000000,2054941,Man of Tai Chi,Keanu Reeves|Tiger Hu Chen|Jeremy Marinas|Stev...,Keanu Reeves,No Rules. No Mercy. Pure Fighting.,"In Beijing, a young martial artist's skill pla...",105,Action|Drama,Universal Pictures|Village Roadshow Pictures|C...,7/4/2013,5.6,2013
120,tt1843866,170000000,714766572,Captain America: The Winter Soldier,Chris Evans|Scarlett Johansson|Sebastian Stan|...,Joe Russo|Anthony Russo,In heroes we trust.,After the cataclysmic events in New York with ...,136,Action|Adventure|Science Fiction,Marvel Studios,3/20/2014,7.6,2014
445,tt1231587,36000000,64572262,Hot Tub Time Machine,John Cusack|Lizzy Caplan|Rob Corddry|Craig Rob...,Steve Pink,Kick some past,A malfunctioning time machine at a ski resort ...,101,Science Fiction|Comedy|Adventure,United Artists|Metro-Goldwyn-Mayer (MGM)|New C...,3/26/2010,5.9,2010
998,tt1024648,44500000,232324128,Argo,Ben Affleck|Bryan Cranston|Alan Arkin|John Goo...,Ben Affleck,The movie was fake. The mission was real.,As the Iranian revolution reaches a boiling po...,120,Drama|Thriller,GK Films|Warner Bros.|Smokehouse Pictures,10/11/2012,7.0,2012
849,tt1564349,37000000,95404397,Dolphin Tale,Morgan Freeman|Ashley Judd|Harry Connick Jr.|N...,Charles Martin Smith,Inspired by the amazing true story of Winter.,A story centered on the friendship between a b...,113,Drama|Family,Alcon Entertainment|Arc Productions,9/23/2011,6.6,2011


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
data['release_date']=pd.to_datetime(data['release_date'])

#Create a profit and loss column (6)
data['profit'] = data['revenue'] - data['budget']

#function to count the number of genres (11)
def genre_counter(genres):
    genres_new = {}
    for line in genres:
        info = line.split('|')
        for genre in info:
            if genre in genres_new:
                genres_new[genre] += 1 
            else:
                genres_new[genre] = 1 
    genre_max = (max(genres_new, key=lambda x: genres_new[x]))
    
    return genre_max    

#add month column (21)
months = {'1': 'Январь', '2': 'Февраль', '3': 'Март', '4': 'Апрель', '5': 'Май',
            '6': 'Июнь', '7': 'Июль', '8': 'Август', '9': 'Сентябрь', '10': 'Октябрь', '11': 'Ноябрь', '12': 'Декабрь'}
data['month'] = data['release_date'].dt.month
data['month'] = data['month'].apply(lambda x: months[str(x)])

#calculate the average length of the studio (25)
def length_calculation(data_mini):
    companies = pd.Series(data_mini['production_companies'].str.cat(sep='|').split('|'))
    companies = companies.value_counts(ascending=False)
    for company in companies.index:
        sum_len = data_mini['length'][data_mini['production_companies'].map(lambda x: True if company in x.split('|') else False)].sum()
        companies[company] = sum_len / companies[company]

    return companies.sort_values(ascending=False)   
    



In [7]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
1210,tt1538403,60000000,90565421,The Mortal Instruments: City of Bones,Lily Collins|Jamie Campbell Bower|Kevin Zegers...,Harald Zwart,You have been chosen,"In New York City, Clary Fray, a seemingly ordi...",130,Action|Adventure|Drama|Mystery|Romance,Constantin Film Produktion|Don Carmody Product...,2013-08-21,6.2,2013,30565421,Август
1014,tt0401729,260000000,284139100,John Carter,Taylor Kitsch|Lynn Collins|Mark Strong|Willem ...,Andrew Stanton,Lost in Our World. Found in Another.,Civil War vet John Carter is transplanted to M...,132,Action|Adventure|Fantasy|Science Fiction,Walt Disney Pictures,2012-03-07,6.0,2012,24139100,Март
1012,tt1707386,61000000,441809770,Les MisÃ©rables,Hugh Jackman|Russell Crowe|Anne Hathaway|Amand...,Tom Hooper,Fight. Dream. Hope. Love.,An adaptation of the successful stage musical ...,157,Drama|Music|Romance,Universal Pictures|Relativity Media|CAMERON MA...,2012-12-18,7.0,2012,380809770,Декабрь
1156,tt0299930,54000000,7266209,Gigli,Ben Affleck|Jennifer Lopez|Justin Bartha|Laini...,Martin Brest,Life doesn't play by the rules.,Gigli is ordered to kidnap the psychologically...,121,Drama,City Light Films|Revolution Studios|Casey Silv...,2003-08-01,3.8,2003,-46733791,Август
1489,tt0430912,70000000,38629478,Basic Instinct 2,Sharon Stone|David Morrissey|Charlotte Ramplin...,Michael Caton-Jones,Everything interesting begins in the mind,Novelist Catherine Tramell is once again in tr...,114,Crime|Mystery|Thriller,Intermedia Films|Kanzaman|C-2 Pictures|IMF Int...,2006-03-29,4.5,2006,-31370522,Март


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [8]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [9]:
# тут пишем ваш код для решения данного вопроса:
budget_max = data[data['budget'] == data['budget'].max()]
print(data['budget'].max())
answer_1 = '5. ' + budget_max['original_title'].values[0] + ' (' + budget_max.imdb_id.values[0] + ')'
print(answer_1)

380000000
5. Pirates of the Caribbean: On Stranger Tides (tt1298650)


ВАРИАНТ 2

In [11]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [11]:
max_time = data[data['runtime'] == data['runtime'].max()]
print(data['runtime'].max())
answers_2 = ('2. ' + max_time.original_title.values[0] + ' (' + max_time.imdb_id.values[0] + ')')
print(answers_2)

214
2. Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [13]:
min_time = data[data['runtime'] == data['runtime'].min()]
print(data['runtime'].min())
answers_3 = ('3. ' + min_time.original_title.values[0] + ' (' + min_time.imdb_id.values[0] + ')')
print(answers_3)

63
3. Winnie the Pooh (tt1449283)


In [14]:
data['runtime'].value_counts()[63]

1

# 4. Какова средняя длительность фильмов?


In [15]:
answers['4'] = '2. 110'

In [16]:
answers_4 = ('2. ' + str(round(data['runtime'].mean(),)))
print(answers_4)

2. 110


# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = '1. 107'

In [18]:
answers_5 = ('1. ' + str(round(data['runtime'].median(),)))
print(answers_5)

1. 107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [19]:
profit_max = data[data['profit'] == data['profit'].max()]
print(data['profit'].max())
answers_6 = ('5. ' + profit_max.original_title.values[0] + ' (' + profit_max.imdb_id.values[0] + ')')
print(answers_6)

2544505847
5. Avatar (tt0499549)


In [20]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [21]:
profit_min = data[data['profit'] == data['profit'].min()]
print(data['profit'].min())
answers_7 = ('5. ' + profit_min.original_title.values[0] + ' (' + profit_min.imdb_id.values[0] + ')')
print(answers_7)

-165710090
5. The Lone Ranger (tt1210819)


In [22]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
profit_count = data[data['profit'] > 0]
answers_8 = '1. ' + str(len(profit_count.index))
print(answers_8)

1. 1478


In [24]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [25]:
grouped_year = data.groupby(['release_year'])['revenue'].max()
year_2008 = data[data['revenue'] == grouped_year[2008]]
answers_9 = ('4. ' + year_2008.original_title.values[0] + ' (' + year_2008.imdb_id.values[0] + ')')
print(answers_9)

4. The Dark Knight (tt0468569)


In [26]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
mini_data = data.query('release_year < 2015 & release_year > 2011')
min_revenue = mini_data[mini_data['profit'] == mini_data.profit.min()]
answers_10 = ('5. ' + min_revenue.original_title.values[0] + ' (' + min_revenue.imdb_id.values[0] + ')')
print(answers_10)

5. The Lone Ranger (tt1210819)


In [28]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [29]:
answers['11'] = '3. Drama'

In [30]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres = data['genres'].str.split('|').tolist() #делаю список из genres и разделяю значения
flat_list = [item for sublist in genres for item in sublist] #избавляюсь от вложенных списков
Count_list = Counter(flat_list) #считаю
answers_11 = ('3. ' + Count_list.most_common(1)[0][0])
print(answers_11)

3. Drama


ВАРИАНТ 2

In [31]:
genres = data['genres']
answers_11 = ('3. ' + genre_counter(genres))
print(answers_11)

3. Drama


ВАРИАНТ 3

In [32]:
genres = data['genres'].tolist()
genres_new = '|'.join(genres).split('|')
genres_count = Counter(genres_new)
answers_11 = ('3. ' + genres_count.most_common(1)[0][0])
print(answers_11)

3. Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = '1. Drama'

In [34]:
genres_profit = data.query('profit > 0')['genres']
answers_12 = ('1. ' + genre_counter(genres_profit))
print(answers_12)

1. Drama


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [35]:
answers['13'] = '5. Peter Jackson'

In [36]:
director_count = pd.Series(data['director'].str.cat(sep='|').split('|'))
director_count = director_count.value_counts(ascending=False)
for director in director_count.index:
    director_count[director] = data['revenue'][data['director'].map(lambda x: True if director in x else False)].sum()
director_revenue = director_count.sort_values(ascending=False)
display(director_revenue.head(5))
answers_13 = ('3. ' + director_revenue.index[0])
print(answers_13)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
dtype: int64

3. Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [38]:
answers['14'] = '3. Robert Rodriguez'

In [37]:
action_genres = data[data.genres.str.contains("Action", na=False)]
director_count = pd.Series(action_genres['director'].str.cat(sep='|').split('|'))
some_director = director_count.value_counts(ascending=False)
display(some_director.sort_values(ascending=False))
answers_14 = ('3. ' + some_director.index[0])
print(answers_14)

Robert Rodriguez         9
Paul W.S. Anderson       7
Michael Bay              7
Antoine Fuqua            6
Ridley Scott             6
                        ..
Paul McGuigan            1
Stephen T. Kay           1
Rian Johnson             1
Gina Prince-Bythewood    1
Jon Avnet                1
Length: 364, dtype: int64

3. Robert Rodriguez


In [39]:
#проверка
print(len(action_genres[action_genres.director.str.contains("Robert Rodriguez", na=False)]))

9


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [40]:
answers['15'] = '3. Chris Hemsworth'

In [41]:
data_12 = data.query('release_year == 2012')
cast = pd.Series(data_12['cast'].str.cat(sep='|').split('|'))
cast = cast.value_counts(ascending=False)
for actor in cast.index:
    cast[actor] = data_12['revenue'][data_12['cast'].map(lambda x: True if actor in x else False)].sum()
cast = cast.sort_values(ascending=False)
display(cast)
answers_15 = ('3. ' + cast.index[0])
print(answers_15)

Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Robert Downey Jr.    1519557910
Mark Ruffalo         1519557910
                        ...    
Paula Patton            3428048
Danny Huston            2106557
Sami Gayle              2106557
Nicolas Cage            2106557
Josh Lucas              2106557
Length: 466, dtype: int64

3. Chris Hemsworth


In [42]:
#проверка
print(data_12[data_12.cast.str.contains("Chris Hemsworth", na=False)]['revenue'].sum())

2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = '3. Matt Damon'

In [44]:
big_budget = data[data['budget'] > data['budget'].mean()]
cast = pd.Series(big_budget['cast'].str.cat(sep='|').split('|'))
cast = cast.value_counts(ascending=False)
print(cast)
answers_16 = ('3. ' + cast.index[0])
print(answers_16)

Matt Damon               18
Adam Sandler             17
Angelina Jolie           16
Samuel L. Jackson        15
Tom Cruise               15
                         ..
Jonas Armstrong           1
Michael Shannon           1
50 Cent                   1
Svetlana Khodchenkova     1
Griffin Gluck             1
Length: 1505, dtype: int64
3. Matt Damon


In [45]:
#проверка
print(big_budget[big_budget.cast.str.contains("Matt Damon", na=False)].count())

imdb_id                 18
budget                  18
revenue                 18
original_title          18
cast                    18
director                18
tagline                 18
overview                18
runtime                 18
genres                  18
production_companies    18
release_date            18
vote_average            18
release_year            18
profit                  18
month                   18
dtype: int64


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [46]:
answers['17'] = '2. Action'

In [47]:
data_nicolas = data[data.cast.str.contains("Nicolas Cage", na=False)]
genres = pd.Series(data_nicolas['genres'].str.cat(sep='|').split('|'))
genres = genres.value_counts(ascending=False)
print(genres)
answers_17 = ('2. ' + genres.index[0])
print(answers_17)

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Animation           3
Mystery             3
History             2
Romance             1
War                 1
Horror              1
dtype: int64
2. Action


In [48]:
#проверка
print(data_nicolas[data_nicolas.genres.str.contains("Action", na=False)].count())

imdb_id                 17
budget                  17
revenue                 17
original_title          17
cast                    17
director                17
tagline                 17
overview                17
runtime                 17
genres                  17
production_companies    17
release_date            17
vote_average            17
release_year            17
profit                  17
month                   17
dtype: int64


# 18. Самый убыточный фильм от Paramount Pictures

In [49]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [50]:
data_ParPic = data[data.production_companies.str.contains("Paramount Pictures", na=False)]
min_profit = data_ParPic[data_ParPic.profit == data_ParPic.profit.min()]
answer_18 = '1. ' + min_profit['original_title'].values[0] + ' (' + min_profit.imdb_id.values[0] + ')'
print(answer_18)

1. K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [51]:
answers['19'] = '5. 2015'

In [52]:
year_data = data.groupby(['release_year'])['revenue'].sum() 
year_data = year_data.sort_values(ascending=False)
print(year_data.head(5))
answers_19 = ('5. ' + str(year_data.index[0]))
print(answers_19)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
Name: revenue, dtype: int64
5. 2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [54]:
answers['20'] = '1. 2014'

In [55]:
data_WBros = data[data.production_companies.str.contains("Warner Bros", na=False)]
data_WBros = data_WBros.groupby(['release_year'])['profit'].sum() 
data_WBros = data_WBros.sort_values(ascending=False)
answers_20 = ('1. ' + str(data_WBros.index[0]))
print(answers_20)

1. 2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [56]:
answers['21'] = '4. Сентябрь'

In [57]:
data_month = data.groupby(data['month'])['original_title'].count()
data_month = data_month.sort_values(ascending=False)
display(data_month)
answers_21 = ('4. ' + str(data_month.index[0]))
print(answers_21)

month
Сентябрь    227
Декабрь     190
Октябрь     186
Август      161
Март        156
Апрель      149
Июнь        147
Ноябрь      146
Июль        142
Май         140
Февраль     135
Январь      110
Name: original_title, dtype: int64

4. Сентябрь


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [58]:
answers['22'] = '2. 450'

In [59]:
data_summer = data[data['month'].isin(['Июнь','Июль','Август'])]
data_summer = data_summer.original_title.count()
display(data_summer)            
answers_22 = ('2. ' + str(data_summer))
print(answers_22)

450

2. 450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [60]:
answers['23'] = '5. Peter Jackson'

In [61]:
data_winter = data[data['month'].isin(['Декабрь','Январь','Февраль'])]
director_count = pd.Series(data_winter['director'].str.cat(sep='|').split('|'))
some_director = director_count.value_counts(ascending=False)     
answers_23 = ('5. ' + some_director.index[0])
print(answers_23)

5. Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [62]:
answers['24'] = '5. Four By Two Productions'

In [63]:
data_mini = data.loc[:, ('original_title','production_companies')]
data_mini['length'] = data['original_title'].str.len()
companies = length_calculation(data_mini)
display(companies)
answers_24 = ('5. ' + companies.index[0])
print(answers_24)

Four By Two Productions       83
Jim Henson Company, The       59
Dos Corazones                 47
Museum Canada Productions     46
Polsky Films                  46
                              ..
Berlanti Productions           3
Everest Entertainment          3
XM2 Productions                2
Ixtlan Productions             2
Global Entertainment Group     2
Length: 1771, dtype: int64

5. Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [64]:
answers['25'] = '3. Midnight Picture Show'

In [65]:
data_mini = data.loc[:, ('overview','production_companies')]
data_mini['length'] = [len(x.split(' ')) for x in data_mini['overview']]
companies = length_calculation(data_mini)
display(companies)
answers_25 = ('3. ' + companies.index[0])
print(answers_25)

Midnight Picture Show                             175
Heineken Branded Entertainment                    161
Room 9 Entertainment                              161
98 MPH Productions                                159
Brookwell-McNamara Entertainment                  156
                                                 ... 
TDJ Enterprises / New Dimensions Entertainment     13
Henceforth                                         13
Projection Pictures                                13
Motion Picture Corporation of America              11
Empire Pictures                                    11
Length: 1771, dtype: int64

3. Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [66]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [67]:
data_average = data['vote_average'].value_counts(bins=10)
max_average = data.loc[(data['vote_average'] > data_average.index[7].left) & (data['vote_average'] <= data_average.index[7].right)]
max_average = max_average.sort_values(['vote_average','original_title'],ascending=False)
films = max_average['original_title']
display(films)

599                                       The Dark Knight
125                                    The Imitation Game
34                                                   Room
118                                          Interstellar
9                                              Inside Out
1183                              The Wolf of Wall Street
872                                           The Pianist
1081        The Lord of the Rings: The Return of the King
138                              The Grand Budapest Hotel
1800                                              Memento
370                                             Inception
119                               Guardians of the Galaxy
128                                             Gone Girl
1191                                     12 Years a Slave
1688                                  There Will Be Blood
155                              The Theory of Everything
1444                                         The Prestige
863           

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [68]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [69]:
pairs = [list(it.combinations(x.split('|'), 2)) for x in data['cast']] #собираем все возможные пары
actor_list = [' & '.join(item) for sublist in pairs for item in sublist] #склеиваем пары символом &
Count_list = Counter(actor_list) 
print(Count_list)
answers_27 = ('5. ' + Count_list.most_common(1)[0][0])
print(answers_27)

Counter({'Daniel Radcliffe & Rupert Grint': 8, 'Daniel Radcliffe & Emma Watson': 8, 'Rupert Grint & Emma Watson': 7, 'Ben Stiller & Owen Wilson': 6, 'Johnny Depp & Helena Bonham Carter': 6, 'Adam Sandler & Kevin James': 5, 'Hugh Jackman & Ian McKellen': 5, 'Kristen Stewart & Robert Pattinson': 5, 'Kristen Stewart & Taylor Lautner': 5, 'Robert Pattinson & Taylor Lautner': 5, 'Jennifer Lawrence & Josh Hutcherson': 4, 'Jennifer Lawrence & Liam Hemsworth': 4, 'Jennifer Lawrence & Woody Harrelson': 4, 'Josh Hutcherson & Liam Hemsworth': 4, 'Josh Hutcherson & Woody Harrelson': 4, 'Liam Hemsworth & Woody Harrelson': 4, 'Tobin Bell & Costas Mandylor': 4, 'Tobin Bell & Betsy Russell': 4, 'Costas Mandylor & Betsy Russell': 4, 'Vin Diesel & Jordana Brewster': 4, 'Paul Walker & Jordana Brewster': 4, 'Mike Myers & Eddie Murphy': 4, 'Mike Myers & Cameron Diaz': 4, 'Eddie Murphy & Cameron Diaz': 4, 'Kristen Stewart & Peter Facinelli': 4, 'Robert Pattinson & Peter Facinelli': 4, 'Taylor Lautner & Pete

ВАРИАНТ 2

In [74]:
casts = pd.Series(data['cast'].str.cat(sep='|').split('|'))
casts = casts.value_counts(ascending=False)
print(casts.head(2))


Samuel L. Jackson    27
Nicolas Cage         27
dtype: int64


# Submission

In [70]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [71]:
# и убедиться что ни чего не пропустил)
len(answers)

27